In [72]:
import pyproj
import shapely
import requests
import geopandas as gpd

geodesic = pyproj.Geod(ellps='WGS84')

def way_id_to_lats_lons(way_id: int) -> tuple[list,list]:
    overpass_url = "http://overpass-api.de/api/interpreter"
        
    overpass_query = f"""
    [out:json];
    way({way_id});
    out geom;
    """

    response = requests.get(overpass_url, params={'data': overpass_query})
    response.raise_for_status()
    data = response.json()

    assert len(data["elements"])==1
    assert len(data["elements"][0]["geometry"])==5

    lats = [e["lat"] for e in data["elements"][0]["geometry"][:-1]]
    lons = [e["lon"] for e in data["elements"][0]["geometry"][:-1]]    

    return lats, lons

In [74]:
lats, lons = way_id_to_lats_lons(1396232705)

In [98]:
for index, (lat, lon) in enumerate(zip(lats, lons)):

    eastToWest = lons[index + 1 if index + 1 != 4 else 0]-lon < 0

    fwd_az, back_az, distance = geodesic.inv(
        lon,
        lat,
        lons[index + 1 if index + 1 != 4 else 0],
        lats[index + 1 if index + 1 != 4 else 0],
    )
    print(f"fwd_az: {fwd_az:.3f}  \tback_az: {back_az:.3f}  \td: {distance:.2f}  \teastToWest: {eastToWest}")


fwd_az: 169.080  	back_az: -10.920  	d: 3.35  	eastToWest: False
fwd_az: 87.966  	back_az: -92.033  	d: 141.28  	eastToWest: False
fwd_az: -12.821  	back_az: 167.179  	d: 3.25  	eastToWest: True
fwd_az: -91.984  	back_az: 88.014  	d: 141.19  	eastToWest: True


In [ ]:
s = gpd.GeoSeries([shapely.geometry.Point(lon, lat) for lat, lon in zip(lats, lons)])

In [48]:
s.set_crs(4326).to_file("check.geojson")

In [49]:
!open .